last updated 03/12/24

This notebook is to check on why current vgg16 runs aren't learning.
this file is importing functions. file was originally a 4c file that did learn as expected.



In [1]:
# imports
import torch

import torch.nn as nn
from torchvision.models import vgg16
import torch.optim.lr_scheduler as lr_scheduler
import torch.optim as optim
from torchvision.models import vgg16
from torch.utils.data import DataLoader
#from torch.Utils.data import DataLoader
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

import numpy as np
import cv2


from datetime import date
from tqdm import tqdm
import pprint
import collections
from IPython.display import clear_output
import time
import random
import cv2

import csv
import json
import pickle
import os

#import wandb

import sys
sys.path.append('../.')

from functions import ImageProcessor,  IDSWDataSetLoader2
from fns4wandb import set_lossfn
from plotting import learning_curve, accuracy_curve, plot_confusion
from modelManagment import get_lin_lay, choose_model2
import copy

#import torch.Utils.data.DataLoader as DataLoader

In [2]:
# file paths
model_type = '4c3l'
optimmy = 'Adam'
Var_WB_sched = "NoSched"


_save_location = f'/its/home/nn268/antvis/antvis/optics/res_big_loop_saves/models/batch/{model_type}/60E/' 
checkpoint_saveloc = f"/its/home/nn268/antvis/antvis/optics/res_big_loop_saves/models/batch/schedulerRuns/{model_type}/{optimmy}/modelCheckPoints/"
# '/its/home/nn268/antvis/antvis/optics/res_big_loop_saves/models/batch/4c/60E//Learning Curve 4c3l_[113, 36]_2025-01-15_Adam_60E_NoSched_run_[113, 36]_0.0001_0_8_MSE.png'
data_path = r'/its/home/nn268/antvis/antvis/optics/AugmentedDS_IDSW/'

gitHASH = 'ac9b2d2910b48bbc4d8cb511b231b637b7edff2a'

device = "cuda:1" if torch.cuda.is_available() else "cpu"

start_epoch = 0
epoch_val = 60  # 150

loadPreTrainedModel = False

sim_type = "run" #(Test or run)

In [3]:
#wandb.login()
d = date.today()

452 144 5/452 *100 = 1%
226 72 5/226 *100 = 2%
113 36 5/113 *100 = 4% -- 2/113 *100= 1.7% ~ 2%
57 18 (56.5,) 5/57 *100 = 8% -- 2/57 *100 = 3.5% ~ 4%. 1/57 = 1.75%
29 9 (28.5,) 5/29 *100 = 17% -- 2/29 *100 = 6.89 ~ 7% 1/28 = 3.57 ~ 4%
15 5 (14.5, 4.5)
8 3 (7.5,2.5)
4, 2 (, 1.5)

In [4]:
# dictionaries                                                                                  * * * *   SETTINGS   * * * *

date = date.today()

model_card_vgg = {'name': 'vgg', 'model': 'vgg16',
                  'f_lin_lay':[200704,#200704,     #129024,#4096,  # (1x229376 and 25088x4096)  1x229376 and 25088x4096) 1x229376 and 25088x4096)
                             200704,      #(16x64512 and 129024x4096)    (16x200704 and 64512x4096)
                             14336,
                             3584,
                             768,
                             4096,
                             4096,
                            ],
                 'idx': 0,
                 'dropout':0.2}


model_card_7c3l = {'name': '7c3l', 'model': '7c3l', 'channels': 3, 'Ks': (3,5),
                  'f_lin_lay':[248832,    # 452 144 # p5
                            59904,      # 226 72 # p5
                            11264,      # 113 36 # p2
                            1536,       # 57 18 # p1
                            172032,           # 29 9
                            172032,          # 15 5
                            172032,         # 8 3
                              ], 
                   'idx': 1,
                  'dropout':0.2}



model_card_4c3l = {'name': '4c3l', 'model': '4c3l', 'channels': 3, 'Ks': (3,5),
                  'f_lin_lay':[539904,# 1055232,#539904,    # 452 144 # p5  (64x539904 and 1055232x100)
                             141056, #141056,    # 226 72 # p5   64x141056 and 267264x100)
                             35840,     # 113 36 # p2   (64x35840 and 304640x100)
                             9984,      # 57 18 # p1 
                             2304,      # 29 9
                             512,       # 15 5
                             256],      # 8 3
                  'idx': 2,
                  'dropout':0.2}      

model_card_3c2l = {'name': '3c2l', 'model': '3c2l', 'channels': 3, 'Ks': (3,5),
                  'f_lin_lay':[1069888,    # 452 144 # p5
                             274688,     #226 72 # p5
                             68096,      # 113 36 # p2
                             17280,      # 57 18 # p1
                             3840,       # 29 9
                             960,        # 15 5
                             256],
                  'idx': 3,
                  'dropout':0.2}       # 8 3

model_card_2c2l = {'name': '2c2l', 'model': '2c2l', 'channels': 3, 'Ks': (3,5),
                  'f_lin_lay':[1055232 , #1032192,# 16883712,#33767424,    # 452 144 # p5 # (1x33767424 and 1055232x100) (1x5276160 and 15828480x100) 1x33767424 and 5276160x100)
                             267264,     #226 72 # p5                   (1x1032192 and 64512x100)
                             64512,#   1032192,#64512,      # 113 36 # p2    ### (16x1055232 and 1032192x100) ###  16x1055232 and 1032192x100)
                             15552,      # 57 18 # p1
                             3072,       # 29 9
                             640,        # 15 5
                             128],
                  'idx': 4,
                  'dropout':0.1}       # 8 3

resolution_card_452144 = {'resolution':[452,144], 'padding':5, 'index':0}
resolution_card_22672 = {'resolution':[226,72], 'padding':5, 'index':1}
resolution_card_11336 = {'resolution':[113,36], 'padding':2, 'index':2}
resolution_card_5715 = {'resolution':[57,18], 'padding':1, 'index':3}

resolution_card_299 = {'resolution':[29,9], 'padding':0, 'index':4} # 
resolution_card_155 = {'resolution':[15,5], 'padding':0, 'index':5}
resolution_card_83 = {'resolution':[8,3], 'padding':0, 'index':6}



resolution_cards = [resolution_card_452144,resolution_card_22672, resolution_card_299,resolution_card_155,resolution_card_83]#[resolution_card_11336, resolution_card_5715,
#resolution_card_299, resolution_card_155, resolution_card_83]#]#resolution_card_452144, 
#resolution_cards = [resolution_card_11336]

#learning_rate_cards = [5e-5, 6e-5, 8e-5]
#learning_rate_cards = [8.21592E-05, 6.62E-05, 6.01E-05, 5.97E-05]
learning_rate_cards=  [1e-4] #[0.1,0.01, 1e-3,1e-4, 1e-5]#, 6e-5, 7e-5, 8e-5]
#wd_cards = [4e-5, 5e-5, 3.00E-05, 2.00E-05]
wd_cards =[0]
scheduler_cards = [0]#, 0.1, 0.2]

seeds = [8,2,4, 42]#,2,3] # 4, 5,6

#model_cards =[model_card_vgg, model_card_7c3l, model_card_4c3l, model_card_3c2l, model_card_2c2l]
model_cards =[model_card_4c3l]

loss_fn_cards = ['MSE'] #,'CrossEntropy' 'MSE'
                        
config = dict({'parameters': 'parameters for big loop run'})
config.update({'model_cards':model_cards})
config.update({'resolution_cards':resolution_cards})
config.update({'learning_rate_cards':learning_rate_cards})
config.update({'wd_cards':wd_cards})
config.update({'scheduler_cards':scheduler_cards})
config.update({'seeds':seeds})
config.update({'loss_fn_cards': loss_fn_cards})


config.update({'batch_size': 64})
config.update({'start_epoch': start_epoch})
config.update({'epochs': epoch_val}) #60

#print(model_card_vgg)
#print('')
#Pp.pprint(Config) # dictionary of dictionaries of lists and lists of dictionaries

In [5]:
from modelManagment import get_lin_lay, choose_model
from functions import get_data
from loop_fns import train_val_batch, test_loop_batch
from fileManagment import save2csv, save2json




def _go(config=None):


    if len(gitHASH) <1:
        print("YOU FORGET THE GIT HASH")
        return
    else:
        print('Git Hash registered')
        pass
        
    #with wandb.init(config=config, project=f"{model_type}. {epoch_val}E. {Var_WB_sched}. {optimmy}_{sim_type}", notes=f"{model_type}. {epoch_val}E. {Var_WB_sched}. {optimmy}_{sim_type}",):
    #config = wandb.config
    start = time.process_time()
        
    for model_idx, model_card in enumerate(config['model_cards']):
        #print("Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
                
        model_name = model_card['model']
        model_index = model_card['idx']
        dropout = model_card['dropout'] 
        for res_idx, resolution_card in enumerate(config['resolution_cards']):
            #print("Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
        
            resolution = resolution_card['resolution']
            pad = resolution_card['padding']
            lin_lay = get_lin_lay(model_card, resolution)
            print('lin lay', lin_lay)
        
            for lr_idx, lr in enumerate(config['learning_rate_cards']):
                for wd_idx, wd_card in enumerate(wd_cards):
                    for sched_idx, scheduler_value in enumerate(config['scheduler_cards']):
                        
                        for seed_idx, seed in enumerate(config['seeds']):
                            seed = seed
                            for lossfn_idx, loss in enumerate(config['loss_fn_cards']):
                                
                                torch.cuda.empty_cache()
                                #print('2')
                                #!nvidia-smi

                                config['batch_size']

                                print('Model: ', str(model_name), f" idx: {model_idx} / {len(config['model_cards'])}")
                                print('resolution: ', str(resolution), f" idx: {res_idx} / {len(config['resolution_cards'])}")
                                print('learning rate: ', str(lr), f" idx: {lr_idx} / {len(config['learning_rate_cards'])}")
                                print('weight decay: ', str(wd_card), f" idx: {wd_idx} / {len(config['wd_cards'])}")
                                print('scheduler: ', str(scheduler_value), f" idx: {sched_idx} / {len(config['scheduler_cards'])}")
                                print('seed: ', str(seed), f" idx: {seed_idx} / {len(config['seeds'])}")
                                print('loss function: ', str(loss), f" idx: {lossfn_idx} / {len(config['loss_fn_cards'])}")
                                print('Batch size: ', config['batch_size'])
                                print('Training epochs: ', config['epochs'])
                                run_start_time = time.process_time()
                                print('start time: ',run_start_time)

                                print(time.process_time() - start)

                                epochs = config['epochs'] #40

                                IP = ImageProcessor(device)

                                #wandb.log({'gitHash':gitHASH})
                                #wandb.log({'Epochs': epochs})
                                
                                save_dict = {'Run' : f"{model_name}_{resolution}_{date}_{optimmy}_{config['epochs']}E_{Var_WB_sched}_{sim_type}",
                                             'Current_Epoch': config['start_epoch'], # this is where i add the start epoch
                                             'start_epoch':config['start_epoch'],
                                             'save_location' : _save_location,
                                             'checkpoint_save_loc': checkpoint_saveloc,
                                             'res': resolution,
                                             'scheduler': Var_WB_sched,
                                             'model': model_name,
                                             'optimiser': optimmy,
                                             'seed':seed}
                                
                                print("model name", model_name, " flinlay: ", lin_lay, " dropout:", dropout)
                                model = choose_model2(model_name, lin_lay, dropout).to(device)
                                
                                # This is where I want to add the new code
                                if loadPreTrainedModel:
                                    dir_pkl = f"/its/home/nn268/antvis/antvis/optics/res_big_loop_saves/models/batch/schedulerRuns/4c/SGD/{Var_WB_sched}/"
                                    pkl_name = f"{model_type}_{resolution}_2024-11-26_SGD_150E_{Var_WB_sched}_{resolution}_0.001_0_{seed}_CrossEntropy.pkl"
                                    #model_pkl = torch.load(dir_pkl+pkl_name)
                                    with open(dir_pkl+pkl_name, 'rb') as f:
                                        model_pkl = pickle.load(f)
                                    #model_pkl = pickle.load(dir_pkl+pkl_name)
                                    model.load_state_dict(model_pkl['model.state_dict'])

                                

                                #print("After model init, Before data loading - Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)

                                x_train, y_train, x_val, y_val, x_test, y_test = get_data(random_seed=seed, file_path=data_path) #, file_path
                                av_lum = IP.new_luminance(x_train)
                                #print("Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
                                
                                train_ds = IDSWDataSetLoader2(x_train, y_train, resolution,pad,av_lum,model_name, device)# av_lum, res,pad,
                                train = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True, drop_last=True) #, num_workers=2

                                
                                test_ds = IDSWDataSetLoader2(x_test, y_test, resolution,pad,av_lum,model_name, device)
                                test = DataLoader(test_ds, batch_size=config['batch_size'], shuffle=True, drop_last=True) #, num_workers=2
                                #print("Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)
                                val_ds = IDSWDataSetLoader2(x_val, y_val, resolution,pad,av_lum,model_name, device)
                                val = DataLoader(val_ds, batch_size=config['batch_size'], shuffle=True, drop_last=True) #, num_workers=2
                                
                                print("After data loading - Current allocated memory (GB):", torch.cuda.memory_allocated() / 1024 ** 3)


                                #print('5')
                                #!nvidia-smi

                                loss_fn = set_lossfn(loss)
                                
                                # set optimizer
                                optimizer = torch.optim.Adam(model.parameters(),lr=lr)# torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)#

                                #wandb.watch(model, loss_fn, log='all', log_freq=2, idx = model_index)

                                loop_run_name = f"{save_dict['Run']}_{resolution}_{lr}_{scheduler_value}_{seed}_{loss}"
     
                                model, save_dict=  train_val_batch(model, train,val, loop_run_name,save_dict, lr, loss_fn,epochs, config['batch_size'], optimizer, scheduler_value, device)

                                test_acc,test_predict_list, y_test = test_loop_batch(model,test, loss_fn, config['batch_size'], device) #model, model_name, X, Y, res, pad, loss_fn, device, num_classes=11
                                
                                #print(test_predict_list)
                                print(' \n train Acc: ', save_dict['t_accuracy_list'][-1])
                                print(' \n val Acc: ', save_dict['v_accuracy_list'][-1])
                                print(' \n test Acc: ', test_acc)
                                
                                save_dict.update({'test_acc': test_acc})
                                save_dict.update({'test_predict': test_predict_list})
                                save_dict.update({'test_labels': list(y_test)})
                                #save_dict.update({'test_loss':test_loss})

                                learning_curve(save_dict['t_loss_list'], save_dict['v_loss_list'], save_location=save_dict['save_location'],run_name=loop_run_name)
                                accuracy_curve(save_dict['t_accuracy_list'], save_dict['v_accuracy_list'],save_location=save_dict['save_location'],run_name=loop_run_name)
                                test_predict_list=test_predict_list#[pred.cpu() for pred in test_predict_list]
                                plot_confusion(predictions= test_predict_list, actual= y_test, title = "Test Confusion matrix", run_name = loop_run_name,save_location =save_dict['save_location'])
                                
                                #wandb.log({'test_acc': test_acc})
                                #wandb.log({'test_predict': test_predict_list})
                                #wandb.log({'test_labels': list(y_test)})
                                #saving
                                diction = {}
                                d = date.today()
                                d=str(d)
                                diction.update({'Date':d})
                                diction.update({'gitHASH':str(gitHASH)})
                                diction.update({'model_name': str(model_name)})
                                diction.update({'loss_fn': str(loss)})
                                diction.update({'lr': str(lr)})
                                diction.update({'wd': str(wd_card)})
                                diction.update({'scheduler value': str(scheduler_value)})
                                diction.update({'seed': str(seed)})
                                diction.update({'resolution': str(resolution)})
                                diction.update({'pad': int(pad)})
                                diction.update({'lin_lay': int(lin_lay)})
                                diction.update({'run time': (time.process_time() - run_start_time)})
                                diction.update(save_dict)
                                
                                save_location = save_dict['save_location']
                                title = save_dict['Run']
                                save2json(diction, loop_run_name, save_location)
                                save2csv(diction, title, save_location)
    
                                diction['model.state_dict'] = model.state_dict() #to('cpu').
    
                                with open(f"{save_location}{loop_run_name}.pkl", 'wb+') as f:
                                    #pickle.dump(diction, f)
                                    torch.save(diction, f)
                                
                                clear_output()
                                
                                print(f' \n END {model_name} {resolution} Run Time: ',time.process_time() - run_start_time)
                                #!nvidia-smi
                                torch.cuda.empty_cache()
    print('Final Run time: ',time.process_time() - start)

In [6]:
_go(config)

 
 END 4c3l [8, 3] Run Time:  69.93893916100205
Final Run time:  30000.820467604


In [7]:
# 12.58 GiB. GPU 0 has a total capacty of 23.65 GiB of which 8.04 GiB is free
23.65-8.04

15.61


pred torch.Size([11])

lab  torch.Size([5, 11])

# 

18/11/24
Added in the checkpoints every 25 epochs
Checked that optimizer = SGD and not optimizer =optimizer = SGD
set total epochs to 150.
added in the creation and saving of LC and AC at each checkpoint.
so checkpoints have a save of the model state dicts and lc and ac so far.

HOWEVER
there still does not look to be any learning... lc are flat and do not change over epochs.
in the pytroch sgd page, the examples use momentum (0.9)
considering running again with  momentum.

hmm. no. 
